# **SD reForge + Zrok por WhiteZ**

**1.1. Copiar y pegar lo siguiente en la terminal** <p>
en la terminal cuando aparezca : Python 3.11 (py311) reinicia tu estancia de sagemaker, ve a kernel>change kernel y selecciona **py311:python**. Luego puedes seguir con la instalación normal
```
conda create -n py311 python=3.11 -y

conda activate py311
conda install ipykernel -y

python -m ipykernel install --user \
       --name py311 \
       --display-name "Python 3.11 (py311)"



**1. BORRA TODOS LOS ARCHIVOS, USALO CON CADA INSTALACIÓN NUEVA** (Puedes ejecutarlo en CPU)

In [ ]:
!rm -r /home/studio-lab-user/.cache
!rm -r /home/studio-lab-user/.conda
!rm -r /home/studio-lab-user/.jupyter
!rm -r /home/studio-lab-user/.sagemaker
!conda remove -n "env_name" --all
!rm -rf /home/studio-lab-user/*

<h4><b>2. INSTALAR Y ACTUALIZAR PYTHON</b></h4> (Puedes ejecutarlo en CPU)

In [ ]:
#instalat ipywidgets
import sys
!{sys.executable} -m pip install -q ipywidgets
#actualizar e instalar paquetes de conda
!conda update -n base -c defaults conda -y
!conda install -c conda-forge ipykernel -y
!conda update pip -y
!conda install conda-forge glib -y
!conda install -c conda-forge pycairo -y
print("Proceso terminado, puedes seguir con las demás celadas")

<h4><b> 3. Decida el repositorio a usar.
Actualizado usa la ultima versión de ReForge.
Mientras que Estable usa una versión fija sin nuevos cambios pero sin errores mayores..</b></h4>

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# ---------- URLs ----------
GITHUB_URL = "https://github.com/Panchovix/stable-diffusion-webui-reForge"
HF_URL     = "https://huggingface.co/datasets/WhiteAiZ/stable-diffusion-webui-reForge"

# ---------- widgets ----------
version = widgets.Dropdown(
    options=["Actualizado", "Estable"],
    value="Estable",
    description="Versión:"
)
instala = widgets.Button(description="Instalar", button_style="info")
out     = widgets.Output()

# ---------- listas de extensiones ----------
ext_full = [
    "gutris1/sd-hub",
    "zanllp/sd-webui-infinite-image-browsing",
    "Coyote-A/ultimate-upscale-for-automatic1111",
    "DominikDoom/a1111-sd-webui-tagcomplete",
    "Bing-su/adetailer",
    "NoCrypt/sd-fast-pnginfo",
    "hako-mikan/sd-webui-supermerger",
    "BlafKing/sd-civitai-browser-plus",
    "viyiviyi/stable-diffusion-webui-zoomimage",
    "gutris1/sd-simple-dimension-preset",
    "pamparamm/sd-perturbed-attention",
    "anxety-solo/sd-civitai-browser-plus",
    "Bing-su/sd-webui-tunnels",
    "Haoming02/sd-forge-couple"
]

ext_stable = [e for e in ext_full if not e.startswith(("Coyote-A", "Bing-su/adetailer", "NoCrypt", "viyiviyi", "gutris1/sd-simple-dimension-preset"))]

# ---------- lógica ----------
def instalar(_):
    with out:
        out.clear_output()
        repo = version.value
        print(f"Clonando repositorio ({repo}) …")
        if repo == "Actualizado":
            get_ipython().system(f'rm -rf /home/studio-lab-user/stable-diffusion-webui-reForge')
            get_ipython().system(f'git clone "{GITHUB_URL}" /home/studio-lab-user/stable-diffusion-webui-reForge')
            # configs adicionales
            get_ipython().system('wget -q https://huggingface.co/datasets/WhiteAiZ/WhiteRF/resolve/main/config.json -O /home/studio-lab-user/stable-diffusion-webui-reForge/config.json')
            get_ipython().system('wget -q https://huggingface.co/datasets/WhiteAiZ/WhiteRF/resolve/main/ui-config.json -O /home/studio-lab-user/stable-diffusion-webui-reForge/ui-config.json')
            get_ipython().system('wget -q https://huggingface.co/datasets/WhiteAiZ/WhiteRF/resolve/main/styles.csv -O /home/studio-lab-user/stable-diffusion-webui-reForge/styles.csv')
        else:
            get_ipython().system(f'rm -rf /home/studio-lab-user/stable-diffusion-webui-reForge')
            get_ipython().system(f'git clone "{HF_URL}" /home/studio-lab-user/stable-diffusion-webui-reForge')

        # ESRGAN
        get_ipython().system('mkdir -p /home/studio-lab-user/stable-diffusion-webui-reForge/models/ESRGAN')
        get_ipython().system('curl -Lo /home/studio-lab-user/stable-diffusion-webui-reForge/models/ESRGAN/4x-UltraSharp.pth https://huggingface.co/Kim2091/AnimeSharp/resolve/main/4x-AnimeSharp.pth')
        get_ipython().system('curl -Lo /home/studio-lab-user/stable-diffusion-webui-reForge/models/ESRGAN/4x-AnimeSharp.pth https://huggingface.co/utnah/esrgan/resolve/main/4x-AnimeSharp.pth')

        # extensiones
        lista = ext_full if repo == "Actualizado" else ext_stable
        get_ipython().system('mkdir -p /home/studio-lab-user/stable-diffusion-webui-reForge/extensions')
        for ext in lista:
            get_ipython().system(f'git clone --depth 1 https://github.com/{ext} /home/studio-lab-user/stable-diffusion-webui-reForge/extensions/{ext.split("/")[1]}')

        # dependencias
        get_ipython().system('pip install -q gdown aria2')
        print("✅ Instalación completada.")

instala.on_click(instalar)

# ---------- mostrar ----------
display(widgets.HBox([version, instala]), out)

<h4><b>4. DESCARGAR MODELOS EN CARPETA TMP DEBES DE EJECUTARLA CADA VEZ QUE INICIES UNA NUEVA SESIÓN (no funcionara con CPU, solo con GPU) </b></h4>

<h4><b>MODELOS NOOBAI-ILLUSTRIOUS</b></h4>

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# ---------- preparación común (solo una vez) ----------
get_ipython().system('rm -rf /home/studio-lab-user/tmp')
get_ipython().system('ln -vs /tmp ~/tmp')
get_ipython().system('mkdir -p /tmp/models')
get_ipython().system('ln -vs /tmp/models /home/studio-lab-user/stable-diffusion-webui-reForge/models/Stable-diffusion/tmp_models')

# ---------- catálogo ----------
modelos = {
    "V-pred – waiSHUFFLENOOBvpred v2.0": (
        "https://huggingface.co/WhiteAiZ/waiSHUFFLENOOB_v20/resolve/main/waiSHUFFLENOOB_vPred20.safetensors",
        "waiSHUFFLENOOB_vPred20.safetensors"
    ),
    "V-pred – PersonalMerge vp05": (
        "https://huggingface.co/datasets/John6666/model-mirror-14/resolve/main/illustriousXLPersonalMerge_vp05testLowsteps.safetensors",
        "PersonalMerge_vp05testLowsteps.safetensors"
    ),
    "V-pred – Rouwei_v080Vpred.safetensors": (
        "https://huggingface.co/WhiteAiZ/RouWei/resolve/main/rouwei_v080Vpred.safetensors",
        "rouwei_v080Vpred.safetensors"
    ),
    "EPS – WAI V15": (
        "https://huggingface.co/WhiteAiZ/WAI-NSFW-illustrious-SDXL-V015/resolve/main/waiNSFWIllustrious_v150.safetensors"
        "waiNSFWIllustrious_v150.safetensors"
    ),
    "EPS – ntrMIXIllustriousXL": (
        "https://huggingface.co/misri/ntrMIXIllustriousXL_xiii/resolve/main/ntrMIXIllustriousXL_xiii.safetensors",
        "ntrMIXIllustriousXL_xiii.safetensors"
    ),
    "EPS – ikastrious": (
        "https://huggingface.co/wsj1995/Checkpoint/resolve/main/874216/2002320/ikastrious_v132bStable.safetensors",
        "ikastrious_v132bStable.safetensors"
    ),
    "EPS – waiSHUFFLENOOB v2.0": (
        "https://huggingface.co/WhiteAiZ/waiSHUFFLENOOB_v20/resolve/main/waiSHUFFLENOOB_v20.safetensors",
        "waiSHUFFLENOOB_v20.safetensors"
    ),
}

# ---------- widgets ----------
dropdown = widgets.Dropdown(
    options=list(modelos.keys()),
    description='Modelo:',
    style={'description_width': 'initial'},
    layout={'width': 'max-content'}
)
boton   = widgets.Button(description='Descargar', button_style='info')
salida  = widgets.Output()

# ---------- lógica ----------
def descargar(_):
    url, nombre = modelos[dropdown.value]
    ruta_tmp = f'/tmp/models/{nombre}'          # disco temporal
    with salida:
        clear_output()
        print(f'⬇️  Descargando {nombre} …')
        get_ipython().system(
            f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M '
            f'"{url}" -o "{ruta_tmp}"'
        )
        print(f'✅ Guardado en {ruta_tmp}  (disco temporal)')

boton.on_click(descargar)

# ---------- mostrar ----------
display(widgets.HBox([dropdown, boton]), salida)

<h4><b>Descargar Modelo Opcional</b></h4>

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from urllib.parse import urlparse
import os

# ---------- preparación ----------
get_ipython().system('mkdir -p /tmp/models')

# ---------- widgets ----------
url_in   = widgets.Text(placeholder='https://…', layout={'width':'60%'})
name_in  = widgets.Text(placeholder='auto',      layout={'width':'30%'})
down_btn = widgets.Button(description='Descargar', button_style='info', icon='download')
out      = widgets.Output()

# ---------- funciones ----------
def sugerir_nombre(change):
    """Rellena el campo nombre cuando el usuario termine de escribir."""
    if not name_in.value.strip():                       # solo si está vacío
        path = urlparse(change['new']).path
        name_in.value = os.path.basename(path) or 'archivo'

def descargar(_):
    url  = url_in.value.strip()
    name = name_in.value.strip() or os.path.basename(urlparse(url).path)
    if not url or not name:
        with out: out.clear_output(); print('❌ Falta URL o nombre.')
        return
    target = f'/tmp/models/{name}'
    cmd    = f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -o "{target}"'
    with out:
        out.clear_output()
        print(f'⬇️  Descargando …\n> {cmd}')
        get_ipython().system(cmd)
        print(f'✅ Listo en {target}')

# ---------- conexiones ----------
url_in.observe(sugerir_nombre, names='value')
down_btn.on_click(descargar)

# ---------- mostrar ----------
display(widgets.HBox([url_in, name_in, down_btn]), out)

**Descargar controlnet union pro max**

In [ ]:
!rm -rf /home/studio-lab-user/tmp/controlnet_models
!mkdir -p /tmp/controlnet_models
!ln -vs /tmp/controlnet_models /home/studio-lab-user/stable-diffusion-webui-reForge/models/ControlNet/
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model_promax.safetensors -o controlnet_union_promax.safetensors
print('DESCARGAS TERMINADAS')

<h4><b> Revisa el espacio disponible</b></h4>

In [ ]:
!df -h | grep -E 'Avail|home'

<h4><b>Ver todas las carpetas y su tamaño</b></h4>

In [ ]:
!du -h --max-depth=1 /home/studio-lab-user/stable-diffusion-webui-reForge | sort -hr

<h4><b>Eliminar cache** (hacerlo despues de cada instalación)</b></h4>

In [ ]:
!rm -r /home/studio-lab-user/.cache

<h4><b>Liberar espacio</b></h4> (Hacerlo despues de cada intalación nueva)

In [ ]:
!conda clean --all -y

<h4><b>Borrar y limpiar carpeta outputs. (recomiendo hacerlo regularmente para no llenar el espacio en disco) </b></h4>

In [ ]:
!rm -rf /home/studio-lab-user/stable-diffusion-webui-reForge/outputs/*

<h5><b>INSTALACION DE ZROK</b></h5>

No olvide pegar su token NGROK a continuación. Crea un token [aquí](https://dashboard.ngrok.com/get-started/your-authtoken)

In [ ]:
%cd /tmp
!wget https://github.com/openziti/zrok/releases/download/v1.1.10/zrok_1.1.10_linux_amd64.tar.gz -q
!mkdir /tmp/zrok
!tar -xf ./zrok*linux*.tar.gz -C /tmp/zrok
!mkdir -p /tmp/zrok/bin && install /tmp/zrok/zrok /tmp/zrok/bin
import os
os.environ['PATH'] = "/tmp/zrok/bin:" + os.environ['PATH']
!zrok version

#Usar solamente en la terminal para crear cuenta en zrok
**export PATH=$PATH:/tmp/zrok/bin <p>**
**zrok invite**

<h4><b> Activa o desactiva tu token de zrok, usar "enable" + tu token para activar y "disable" sin el token para desactivar</b></h4>

In [ ]:
!zrok enable tu_token

<h5><b>LANZAR SD CON ZROK</b></h5>

In [ ]:
%cd ~/stable-diffusion-webui-reForge
from multiprocessing import Process
import time

def run_command1():
    time.sleep(15)
    !zrok share public http://localhost:7860 --headless

def run_command2():
    time.sleep(2)
    !pip install torch==2.7.1
    !python launch.py --no-hashing  --disable-console-progressbars --disable-xformers --opt-sdp-attention --enable-insecure-extension-access --theme dark --gradio-queue --no-download-sd-model

if __name__ == '__main__':
    p1 = Process(target=run_command1)
    p2 = Process(target=run_command2)
    p1.start()
    p2.start()
    p1.join()
    p2.join()

<h5><b>LANZAR SD CON NGROK</b></h5>

In [ ]:
#No se olvide de pegar su token de NGROK aquí
%cd ~/stable-diffusion-webui-reForge
!python launch.py --ngrok coloca_tu_token_aqui --no-hashing  --disable-console-progressbars disable-xformers --opt-sdp-attention --enable-insecure-extension-access --theme dark --gradio-queue --no-download-sd-model

<h4><b>🖼️ Descargar carpeta Output</b></h4>

In [ ]:
#download and compress all outputs
import os
import zipfile
def zip_folder(input_folder, output_folder, max_size_mb=20):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    zip_number = 1
    current_zip_size = 0
    current_zip_name = os.path.join(output_folder, f"part_{zip_number}.zip")
    with zipfile.ZipFile(current_zip_name, 'w', zipfile.ZIP_DEFLATED) as current_zip:
        for root, _, files in os.walk(input_folder):
            for file in files:
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)
                # Check if adding the current file exceeds the max size
                if current_zip_size + file_size > max_size_mb * 1024 * 1024:
                    # Close the current zip file
                    current_zip.close()
                    # Start a new zip file
                    zip_number += 1
                    current_zip_name = os.path.join(output_folder, f"part_{zip_number}.zip")
                    current_zip = zipfile.ZipFile(current_zip_name, 'w', zipfile.ZIP_DEFLATED)
                    current_zip_size = 0
                # Add the file to the current zip
                current_zip.write(file_path, os.path.relpath(file_path, input_folder))
                current_zip_size += file_size
# Specify the input folder to be zipped
input_folder = 'stable-diffusion-webui-reForge/outputs'  # Replace with the path to your input folder
# Specify the output folder where zip files will be saved
output_folder = 'outputs'  # Replace with the path to your output folder
# Specify the maximum size for each zip file in MB
max_size_mb = 199
# Create zip files with a maximum size of 20MB
zip_folder(input_folder, output_folder, max_size_mb)